# Generate configuration files for each chip

The experiment contains the following chips:

* Chip1 (k1)
* Chip2 (k1)

From the template configuration file, we need to change the following for each chip: 
- image[base_path] -> append Chip# subdirectory
- image[names] -> Chip#

And add in the following from generator
- image[well_start_image]
- image[well_start_xy]
- well_mask

## Generate configuration files

In [1]:
import yaml 
import copy
from os import path, mkdir

In [2]:
with open('config.yml', 'r') as ymlfile:
    template = yaml.load(ymlfile)

with open('generator.yml', 'r') as ymlfile:
    generator = yaml.load(ymlfile)
    
print yaml.dump(template)
print yaml.dump(generator)

barcodes:
  cluster:
    eps: 0.01
    min_samples: 10
    offset: [600, 600, 600]
    points_to_cluster: 2000
  path: ../Notes.xlsx
image:
  base_path: /Volumes/Seagate Backup Plus Drive/20180418_focalalone/Image Data/
  bugs: 0
  dyes: [1, 2, 3]
  names: {premerge: Chip1_t0, t0: Chip1_t0, t1: Chip1_t1, t2: Chip1_t2, t3: Chip1_t3,
    t4: Chip1_t4, t5: Chip1_t5}
  overlap: 0.05
  pixel_size: 6.5
  size: 1024
  well_start_image: [1, 1]
  well_start_xy: [540, 140]
well_mask:
  filename: kchip/well_mask.tif
  pixel_size: 8.33625
  well_start_xy: [850, 522]

Chip1:
  image:
    well_start_image: [1, 1]
    well_start_xy: [350, 350]
  well_mask:
    filename: ../Well Masks/k1_mask.tif
    pixel_size: 8.33625
    well_start_xy: [182, 202]
Chip2:
  image:
    well_start_image: [1, 1]
    well_start_xy: [525, 320]
  well_mask:
    filename: ../Well Masks/k1_mask.tif
    pixel_size: 8.33625
    well_start_xy: [182, 202]



In [3]:
# Create base analysis directory
base = '/Users/akulesa/Desktop/20180418_focalalone/'

if not path.exists(base):
    mkdir(base)
    
# Create directories
for chip in generator.keys():

    if not path.exists(path.join(base,chip)):
        mkdir(path.join(base,chip))

    if not path.exists(path.join(base,chip)):
        mkdir(path.join(base,chip))

In [4]:
for chip in generator.keys():

    config = copy.deepcopy(template)

    try: 
        config['image']['base_path'] = path.join(config['image']['base_path'],chip)

        for name in config['image']['names'].keys():
            config['image']['names'][name] = config['image']['names'][name].replace('Chip1',chip)

        config['image']['well_start_image'] = generator[chip]['image']['well_start_image']
        config['image']['well_start_xy'] = generator[chip]['image']['well_start_xy']
        config['well_mask'] = generator[chip]['well_mask']

        with open(path.join(base,chip,'config.yml'), 'w') as outfile:
            yaml.dump(config, outfile,default_flow_style=False)
    except:
        print 'Error on:', chip

## Setup jupyter notebooks for each chip

Use Analysis_template.ipynb as template

In [ ]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

def create_notebook(template):
    ''' Create output notebook from template '''

    with open(template) as f:
        nb = nbformat.read(f, as_version=4)

    return nb

def execute_notebook(nb):
    ''' Execute notebook (returns in place) kernel=python2.7'''
    ep = ExecutePreprocessor(timeout=600, kernel_name='python2.7')
    ep.preprocess(nb,{'metadata': {'path': './'}})

def write_notebook(nb,output_path):
    ''' Write notebok file '''
    with open(output_path, 'wt') as f:
        nbformat.write(nb, f)


In [ ]:
template = '/Users/akulesa/Desktop/180318/Analysis/Analysis_Template.ipynb'
from os import chdir

for chip in generator.keys():
    print chip

    print 'Creating...'
    # Create from template
    nb = create_notebook(template)

    print 'Moving to... '
    chdir(path.join(base,chip))

    print 'Executing...'

    try: 
        # Execute
        execute_notebook(nb)

    except Exception as e:
        print 'Error executing.'
        print e

    print 'Writing...'
    # Write
    outfile = 'Analysis.ipynb'
    write_notebook(nb,outfile)